# Project 1 - Data Warehousing | Marco Rossini (s291482)

## Data import and connection with the database

In [ ]:
import pandas as pd

### Import the input tables

Read from URL

In [ ]:
dim_restaurant=pd.read_csv("https://marcorossini.altervista.org/projects/master/advanced-databases/project-work-1/dim_restaurant.csv")
dim_time=pd.read_csv("https://marcorossini.altervista.org/projects/master/advanced-databases/project-work-1/dim_time.csv")
dim_category=pd.read_csv("https://marcorossini.altervista.org/projects/master/advanced-databases/project-work-1/dim_category.csv")
fact_delivery=pd.read_csv("https://marcorossini.altervista.org/projects/master/advanced-databases/project-work-1/fact_delivery.csv")

### Create a local SQLite database.

The `create_engine()` function takes the database as one argument. We indicate the URL as the first positional argument with connection arguments. Using the code given below, we can create a database.


In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///deliveries.db')

### Write records stored in the dataframe to the SQL database.


After creating our engine, we need to define and create our tables. We use the `to_sql` function of pandas.

In [ ]:
dim_restaurant.to_sql('dim_restaurant', con=engine, if_exists="replace", index=False)
dim_time.to_sql("dim_time", con=engine, if_exists="replace", index=False)
dim_category.to_sql("dim_category", con=engine, if_exists="replace", index=False)
fact_delivery.to_sql("fact_delivery", con=engine, if_exists="replace", index=False)

In [ ]:
dim_restaurant.head()

In [ ]:
dim_time.head()

In [ ]:
dim_category.head()

In [ ]:
fact_delivery.head()


## Querying the Data Warehouse

### Query A

*\"For each day, select the total revenue and the average revenue per delivery. Sort the result by date.\"*

In [ ]:
query_a= "SELECT Date, SUM(TotalRevenue) as TotalRevenue, SUM(TotalRevenue) / SUM(NumberOfDeliveries) as AverageRevenuePerDelivery \
          FROM fact_delivery F, dim_time T \
          WHERE F.TimeID = T.TimeID \
          GROUP BY Date \
          ORDER BY Date"

pd.read_sql_query(query_a, engine)

### Query B


*\"Select the yearly revenue and the total number of deliveries for each restaurant. Sort the results by descending yearly revenue.\"*

In [ ]:
query_b= "SELECT Restaurant, Year, SUM(TotalRevenue) as YearlyRevenue, SUM(NumberOfDeliveries) as TotalNumberOfDeliveries \
          FROM fact_delivery F, dim_time T, dim_restaurant R \
          WHERE F.RestaurantID = R.RestaurantID AND F.TimeID = T.TimeID \
          GROUP BY Restaurant, Year \
          ORDER BY YearlyRevenue DESC"

pd.read_sql_query(query_b, engine)

### Query C


*\"Separately for each transport mode and year, select the total number of deliveries and the average time for delivery.\"*

In [ ]:
query_c= "SELECT TransportMode, Year, SUM(NumberOfDeliveries) as TotalNumberOfDeliveries, SUM(TotalDeliveryTime) / SUM(NumberOfDeliveries) as AverageDeliveryTime \
          FROM fact_delivery F, dim_time T \
          WHERE F.TimeID = T.TimeID \
          GROUP BY TransportMode, Year"

pd.read_sql_query(query_c, engine)

### Query D


*\"Consider only the deliveries with “bike” as transport mode. Separately for each month and restaurant, select the total revenue and the average delivery time.\"*

In [ ]:
query_d= "SELECT Month, Restaurant, SUM(TotalRevenue) as TotalRevenue, SUM(TotalDeliveryTime) / SUM(NumberOfDeliveries) as AverageDeliveryTime \
          FROM fact_delivery F, dim_time T, dim_restaurant R \
          WHERE F.TimeID = T.TimeID AND F.RestaurantID = R.RestaurantID AND F.TransportMode = 'Bike' \
          GROUP BY Month, Restaurant"

pd.read_sql_query(query_d, engine)

### Query E


*\"Separately for date and transport mode, select the total revenue and the maximum delivery time.\"*

In [ ]:
query_e= "SELECT Date, TransportMode, SUM(TotalRevenue) as TotalRevenue, MAX(TotalDeliveryTime) as MaxDeliveryTime \
          FROM fact_delivery F, dim_time T \
          WHERE F.TimeID = T.TimeID \
          GROUP BY Date, TransportMode"

pd.read_sql_query(query_e, engine)

### Query F


*\"Separately for each month, select the total revenue and the average daily revenue.\"*

In [ ]:
query_f= "SELECT Month, SUM(TotalRevenue) as TotalRevenue, SUM(TotalRevenue) / COUNT(DISTINCT Date) as AverageDailyRevenue \
          FROM fact_delivery F, dim_time T \
          WHERE F.TimeID = T.TimeID \
          GROUP BY Month"

pd.read_sql_query(query_f, engine)

## Join for Google Data Studio

In [ ]:
query_join = "SELECT Restaurant, Address, City, Province, Region, Date, Weekday, Holiday, Month, Semester, Year, Category, PaymentMethod, TransportMode, TotalRevenue, TotalDeliveryTime, NumberOfDeliveries \
              FROM fact_delivery F, dim_restaurant R, dim_time T, dim_category C \
              WHERE F.RestaurantID = R.RestaurantID AND F.CategoryID = C.CategoryID AND F.TimeID = T.TimeID"

In [ ]:
df=pd.read_sql_query(query_join, engine)

In [ ]:
df.head()

In [ ]:
df.to_csv("./deliveries_joined.csv", index=False)

The joined table is used in Google Data Studio for visualization purposes. The related report is available at this [link](https://datastudio.google.com/reporting/e0fd2c8b-c713-4cf3-a9fe-705bc2f8ca6a).